In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from src.shared.faces import make_cube_faces
from src.shared.sides import (
    sphered_vertices,
    to_vertices,
    to_stacked,
)

from scripts.geometry import (
    compute_vertex_normals, 
    compute_face_normals,
)

import meshplot

In [10]:
# Parametrization of a onion
class Onion(nn.Module):
    def __init__(self):
        super(Onion, self).__init__()
        # Center
        self.root = nn.Parameter(torch.zeros(3))
        #self.
        #self.register_buffer('phi', phi)
        
    def forward(self):
        # ratios = torch.sigmoid(center_parameters) * scale
        # vertices = center + center_normals * ratios
        # for layer in layers:
        #     vertices = torch.expand(vertices)
        #     normals = calculate_normals(vertices, faces)
        #     ratios = torch.sigmoid(layer_parameters) * layer_scale
        #     vertices = vertices + normals * ratios
        return vertices
    

onion = Onion()
onion

Onion()

In [9]:
faces = make_cube_faces(2)
faces.shape

torch.Size([44, 3])

In [12]:
sphered_vertices(2, 0.5).shape

torch.Size([6, 3, 2, 2])

In [52]:
class Root(nn.Module):
    def __init__(self, n, scale):
        super(Root, self).__init__()
        self.scale = scale
        self.center = nn.Parameter(torch.zeros(6, 3, 1, 1))
        self.ratios = nn.Parameter(torch.randn(6, 3, n, n) * 0.05)
        self.register_buffer('normals', sphered_vertices(n, 1))
        self.register_buffer('faces', make_cube_faces(n).int())
        
    def forward(self):
        vertices = self.center + self.normals * torch.sigmoid(self.ratios) * self.scale
        return vertices
    
root = Root(8, 1) 

meshplot.plot(to_vertices(root()).detach().cpu().numpy(), root.faces.numpy());

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0163567…

In [36]:
to_vertices(root()).shape

torch.Size([24, 3])

In [60]:
class Layer(nn.Module):
    def __init__(self, n, scale):
        super(Layer, self).__init__()
        self.n = n
        self.scale = scale        
        self.ratios = nn.Parameter(torch.zeros(6, 3, n, n))        
        self.register_buffer('faces', make_cube_faces(n).int())
        
    def forward(self, vertices):
        vert = F.interpolate(vertices, size=(self.n, self.n), mode='bilinear', align_corners=True)
        v_f = to_vertices(vert)
        face_normals = compute_face_normals(v_f, self.faces)
        normals = compute_vertex_normals(v_f, self.faces, face_normals)
        normals = to_stacked(normals)
        
        vertices = vert + normals * torch.sigmoid(self.ratios) * self.scale
        return vertices

layer = Layer(2, 0.125) 
layer(root())

meshplot.plot(to_vertices(layer(root())).detach().cpu().numpy(), layer.faces.numpy());

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.015286…